In [1]:
#%%

import time
import json
import numpy as np
import pandas as pd
from pandas import json_normalize

import datetime

import seaborn as sns
import matplotlib.pyplot as plt


from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error

import warnings
warnings.filterwarnings('ignore')

#Scatterplot and Correlation Matrix - Kaustubh
#Frequency Tables - Jonathan and Stephanie
#Naiyu - histograms

In [3]:
#%%
def load_df(csv_path, nrows = None):
    json_cols = ['device', 'geoNetwork', 'totals', 'trafficSource']
    df = pd.read_csv(csv_path,
                     #converters are dict of functions for converting values in certain columns. Keys can either be integers or column labels.
                     #json.loads() method can be used to parse a valid JSON string and convert it into a Python Dictionary.
                     #It is mainly used for deserializing native string, byte, or byte array which consists of JSON data into Python Dictionary.
                     converters = {col: json.loads for col in json_cols},                                                                         
                         dtype = {'fullVisitorId': 'str'}, # Important!!
                         nrows = nrows)
    for col in json_cols:
        # for each column, flatten data frame such that the values of a single col are spread in different cols
        # This will use subcol as names of flat_col.columns
        flat_col = json_normalize(df[col])
        # Name the columns in this flatten data frame as col.subcol for tracability
        flat_col.columns = [f"{col}.{subcol}" for subcol in flat_col.columns]
        # Drop the json_col and instead add the new flat_col
        df = df.drop(col, axis = 1).merge(flat_col, right_index = True, left_index = True)
    return df


csv_train_path = 'train.csv'
csv_test_path = 'test.csv'

train = load_df(csv_train_path, nrows = 100000)
# test = load_df(csv_test_path, nrows = 30000)
train.shape
# %%

(100000, 55)

In [4]:
train

,channelGrouping,date,fullVisitorId,sessionId,socialEngagementType,visitId,visitNumber,visitStartTime,device.browser,device.browserVersion,...,trafficSource.adwordsClickInfo.criteriaParameters,trafficSource.isTrueDirect,trafficSource.referralPath,trafficSource.adwordsClickInfo.page,trafficSource.adwordsClickInfo.slot,trafficSource.adwordsClickInfo.gclId,trafficSource.adwordsClickInfo.adNetworkType,trafficSource.adwordsClickInfo.isVideoAd,trafficSource.adContent,trafficSource.campaignCode
0,Organic Search,20160902,1131660440785968503,1131660440785968503_1472830385,Not Socially Engaged,1472830385,1,1472830385,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Organic Search,20160902,377306020877927890,377306020877927890_1472880147,Not Socially Engaged,1472880147,1,1472880147,Firefox,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Organic Search,20160902,3895546263509774583,3895546263509774583_1472865386,Not Socially Engaged,1472865386,1,1472865386,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Organic Search,20160902,4763447161404445595,4763447161404445595_1472881213,Not Socially Engaged,1472881213,1,1472881213,UC Browser,not available in demo dataset,...,not available in demo dataset,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Organic Search,20160902,27294437909732085,27294437909732085_1472822600,Not Socially Engaged,1472822600,2,1472822600,Chrome,not available in demo dataset,...,not available in demo dataset,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Social,20161030,06201955442296075,06201955442296075_1477892744,Not Socially Engaged,1477892744,1,1477892744,Safari,not available in demo dataset,...,not available in demo dataset,NaN,/yt/about/th/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99996,Social,20161030,3638854205116467643,3638854205116467643_1477840657,Not Socially Engaged,1477840657,1,1477840657,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,/yt/about/zh-TW/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99997,Social,20161030,5058770420082063061,5058770420082063061_1477889694,Not Socially Engaged,1477889694,1,1477889694,Chrome,not available in demo dataset,...,not available in demo dataset,NaN,/yt/about/vi/,NaN,NaN,NaN,NaN,NaN,NaN,NaN
99998,Social,20161030,5749057926198995016,5749057926198995016_1477842029,Not Socially Engaged,1477842029,1,1477842029,Safari,not available in demo dataset,...,not available in demo dataset,NaN,/yt/about/fr/,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
len(train["device.browserVersion"].unique())
train.date.dtype == 'int64'

True

In [6]:
train.dtypes

channelGrouping                                      object
date                                                  int64
fullVisitorId                                        object
sessionId                                            object
socialEngagementType                                 object
visitId                                               int64
visitNumber                                           int64
visitStartTime                                        int64
device.browser                                       object
device.browserVersion                                object
device.browserSize                                   object
device.operatingSystem                               object
device.operatingSystemVersion                        object
device.isMobile                                        bool
device.mobileDeviceBranding                          object
device.mobileDeviceModel                             object
device.mobileInputSelector              

In [7]:
cat_vars = []
for col in train.columns:
    if train[col].dtype == 'O':
        if len(train[col].value_counts()) > 1:
            cat_vars.append(col)

In [8]:
cat_vars

['channelGrouping',
 'fullVisitorId',
 'sessionId',
 'device.browser',
 'device.operatingSystem',
 'device.deviceCategory',
 'geoNetwork.continent',
 'geoNetwork.subContinent',
 'geoNetwork.country',
 'geoNetwork.region',
 'geoNetwork.metro',
 'geoNetwork.city',
 'geoNetwork.networkDomain',
 'totals.hits',
 'totals.pageviews',
 'totals.transactionRevenue',
 'trafficSource.campaign',
 'trafficSource.source',
 'trafficSource.medium',
 'trafficSource.keyword',
 'trafficSource.referralPath',
 'trafficSource.adwordsClickInfo.page',
 'trafficSource.adwordsClickInfo.slot',
 'trafficSource.adwordsClickInfo.gclId',
 'trafficSource.adwordsClickInfo.adNetworkType',
 'trafficSource.adContent']

In [9]:
for i in cat_vars:
    print(pd.crosstab(index=train[i], columns='count') .sort_values("count", ascending = False))

col_0            count
channelGrouping       
Organic Search   42019
Social           24378
Direct           15814
Referral         12095
Paid Search       3022
Affiliates        1900
Display            768
(Other)              4
col_0                count
fullVisitorId             
1957458976293878100     44
0824839726118485274     26
3269834865385146569     23
3608475193341679870     21
4038076683036146727     19
...                    ...
3504260133191015550      1
3504190896738931466      1
3504105867871633072      1
3504095512782487176      1
9999799804624735268      1

[89213 rows x 1 columns]
col_0                           count
sessionId                            
0387871673225482446_1480492793      2
7817946123013268752_1480492790      2
946085548949296102_1480492063       2
7980925080669177483_1498285182      2
0996739878901151740_1497682650      2
...                               ...
3376391763828760484_1478837572      1
3376334945486588027_1481325414      1
3376183746916

col_0                      count
totals.transactionRevenue       
16990000                      28
18990000                      25
33590000                      22
44790000                      20
13590000                      18
...                          ...
298270000                      1
298760000                      1
29880000                       1
2990000                        1
35080000                       1

[969 rows x 1 columns]
col_0                                            count
trafficSource.campaign                                
(not set)                                        95442
Data Share Promo                                  1901
AW - Dynamic Search Ads Whole Site                1637
AW - Accessories                                   921
test-liyuhz                                         56
Retail (DO NOT EDIT owners nophakun and tianyu)     27
AW - Apparel                                         9
AW - Electronics                                    

In [10]:
# Notes on variables

# There are multiple observations for some vistors and devices
# in order for observations to be indep, need to remove multiple observations


# Identified vars that had multiple values and were not mostly missing

# ---- Usuable cat vars ----
# channelGrouping
# device.browser
# device.operatingSystem
# device.deviceCategory
# geoNetwork.continent
# geoNetwork.subContinent
# geoNetwork.country
# totals.hits
# totals.pageviews
# totals.transactionRevenue
# trafficSource.source
# trafficSource.medium
# trafficSource.adContent

In [12]:
from tabulate import tabulate

In [37]:
usuable_cat_vars = ["channelGrouping", "device.browser", "device.operatingSystem", "device.deviceCategory",
                    "geoNetwork.continent", "geoNetwork.subContinent", "geoNetwork.country", "totals.hits",
                    "totals.pageviews", "totals.transactionRevenue", "trafficSource.source", "trafficSource.medium",
                    "trafficSource.adContent"]

# for var in usuable_cat_vars:
#     print((pd.crosstab(index=train[var], columns='count') .sort_values("count", ascending = False)).to_markdown)


from IPython.core import display as ICD
for var in usuable_cat_vars:
    ICD.display((pd.crosstab(index=train[var], columns='count') .sort_values("count", ascending = False)))

col_0,count
channelGrouping,
Organic Search,42019
Social,24378
Direct,15814
Referral,12095
Paid Search,3022
Affiliates,1900
Display,768
(Other),4


col_0,count
device.browser,
Chrome,68984
Safari,20467
Firefox,4004
Internet Explorer,2080
Edge,1063
Safari (in-app),743
Android Webview,715
Opera Mini,631
Opera,525


col_0,count
device.operatingSystem,
Windows,38001
Macintosh,29089
Android,13323
iOS,11718
Linux,4114
Chrome OS,3055
(not set),458
Windows Phone,153
BlackBerry,28


col_0,count
device.deviceCategory,
desktop,74135
mobile,22635
tablet,3230


col_0,count
geoNetwork.continent,
Americas,50435
Asia,24661
Europe,21601
Africa,1600
Oceania,1578
(not set),125


...

col_0,count
totals.pageviews,
1,49087
2,16708
3,8223
4,4987
5,3704
...,...
119,1
120,1
126,1


col_0,count
totals.transactionRevenue,
16990000,28
18990000,25
33590000,22
44790000,20
13590000,18
...,...
298270000,1
298760000,1
29880000,1


col_0,count
trafficSource.source,
google,44448
youtube.com,23060
(direct),15814
mall.googleplex.com,7895
Partners,1901
...,...
google.gatewaycdi.com,1
google.es,1
google.com.vn,1


col_0,count
trafficSource.medium,
organic,42019
referral,36473
(none),15814
cpc,3022
affiliate,1900
cpm,768
(not set),4


col_0,count
trafficSource.adContent,
Google Merchandise Collection,662
Google Online Store,148
Display Ad created 3/11/14,114
Full auto ad IMAGE ONLY,98
Display Ad created 3/11/15,56
Ad from 12/13/16,50
{KeyWord:Google Merchandise},28
Display Ad created 11/17/14,27
{KeyWord:Google Brand Items},27


In [30]:
pd.crosstab(index=train["channelGrouping"], columns='count') .sort_values("count", ascending = False)

col_0,count
channelGrouping,
Organic Search,42019
Social,24378
Direct,15814
Referral,12095
Paid Search,3022
Affiliates,1900
Display,768
(Other),4


In [39]:
train.to_csv('train_s.csv')